In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import cv2

In [2]:
label_data = pd.read_csv('kaggle/train_study_level.csv')

In [3]:
label_data.shape

(6054, 5)

In [4]:
label2 = pd.read_csv('kaggle/train_image_level.csv')

In [5]:
label2.shape

(6334, 4)

In [6]:
label2.head(2)

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed


In [7]:
label2['StudyInstanceUID'] = label2['StudyInstanceUID'].apply(lambda x : x+'_study')

In [8]:
label_data = label_data.merge(label2,left_on='id',right_on='StudyInstanceUID',how='left')

In [9]:
label_data.head(2)

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
0,00086460a852_study,0,1,0,0,65761e66de9f_image,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852_study
1,000c9c05fd14_study,0,0,0,1,51759b5579bc_image,NaN,none 1 0 0 1 1,000c9c05fd14_study


In [10]:
label_data['id_x'].value_counts(ascending=False)

0fd2db233deb_study    9
a7335b2f9815_study    7
8943d1d85097_study    6
a0254bf8a96e_study    5
a4e94133d95a_study    5
                     ..
0adfd8d6f010_study    1
6703e8f113c4_study    1
bc2b65a2e0e0_study    1
eb30f3452fc6_study    1
c05506118d89_study    1
Name: id_x, Length: 6054, dtype: int64

In [11]:
label_data[label_data['id_x']=='0fd2db233deb_study']

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
375,0fd2db233deb_study,0,0,1,0,04cc2f7f4c4b_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
376,0fd2db233deb_study,0,0,1,0,05c063f5cef5_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
377,0fd2db233deb_study,0,0,1,0,156cb1f5c689_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
378,0fd2db233deb_study,0,0,1,0,26f643772090_image,"[{'x': 561.30269, 'y': 163.28797, 'width': 883...",opacity 1 561.30269 163.28797 1445.02377 1495....,0fd2db233deb_study
379,0fd2db233deb_study,0,0,1,0,4c414b793562_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
380,0fd2db233deb_study,0,0,1,0,a5a364383f34_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
381,0fd2db233deb_study,0,0,1,0,b121806162c3_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
382,0fd2db233deb_study,0,0,1,0,bee62c601ae9_image,NaN,none 1 0 0 1 1,0fd2db233deb_study
383,0fd2db233deb_study,0,0,1,0,c6e92e59a0ae_image,NaN,none 1 0 0 1 1,0fd2db233deb_study


In [12]:
label_data[label_data['id_y']=='04cc2f7f4c4b_image']

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
375,0fd2db233deb_study,0,0,1,0,04cc2f7f4c4b_image,NaN,none 1 0 0 1 1,0fd2db233deb_study


In [13]:
label_data.id_y.value_counts(ascending=False)

796c8b8701bf_image    1
7e73965d19a5_image    1
3b9431421ee9_image    1
795ceeb25970_image    1
089bd77c8c10_image    1
                     ..
4aac2ad7b8fa_image    1
c80d2df370e8_image    1
de32dd96114c_image    1
ecc2572103d7_image    1
78148afc0575_image    1
Name: id_y, Length: 6334, dtype: int64

In [14]:
label_data.drop_duplicates(subset=['id_x'],inplace=True)

In [15]:
label_data.shape

(6054, 9)

In [16]:
label_data.head(2)

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
0,00086460a852_study,0,1,0,0,65761e66de9f_image,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852_study
1,000c9c05fd14_study,0,0,0,1,51759b5579bc_image,NaN,none 1 0 0 1 1,000c9c05fd14_study


In [17]:
label_data.columns

Index(['id_x', 'Negative for Pneumonia', 'Typical Appearance',
       'Indeterminate Appearance', 'Atypical Appearance', 'id_y', 'boxes',
       'label', 'StudyInstanceUID'],
      dtype='object')

In [18]:
for i in ['Negative for Pneumonia', 'Typical Appearance','Indeterminate Appearance', 'Atypical Appearance']:
    label_data[i] = label_data[i].replace(1,i) 

In [19]:
label_data['target'] = label_data['Negative for Pneumonia'].astype('str') + label_data['Atypical Appearance'].astype('str') + \
                        label_data['Typical Appearance'].astype('str') + label_data['Indeterminate Appearance'].astype('str')

In [20]:
label_data['target'] = label_data['target'].apply(lambda x: re.sub('0','',x))

In [21]:
label_data['target'].value_counts()

Typical Appearance          2855
Negative for Pneumonia      1676
Indeterminate Appearance    1049
Atypical Appearance          474
Name: target, dtype: int64

In [22]:
label_data.sample()

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID,target
1020,29b7fce85c4c_study,0,Typical Appearance,0,0,ab55abb953d1_image,NaN,none 1 0 0 1 1,29b7fce85c4c_study,Typical Appearance


In [23]:
label_data['id_y'] = label_data['id_y'].apply(lambda x: re.sub('_image','.jpg',x))

In [24]:
label_data['path'] = 'gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train/' + label_data['id_y'].astype('str')

In [25]:
label_data.head(2)

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID,target,path
0,00086460a852_study,0,Typical Appearance,0,0,65761e66de9f.jpg,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852_study,Typical Appearance,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...
1,000c9c05fd14_study,0,0,0,Atypical Appearance,51759b5579bc.jpg,NaN,none 1 0 0 1 1,000c9c05fd14_study,Atypical Appearance,gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/train...


In [31]:
label_data.target.value_counts()

Typical_Appearance          2855
Negative_for_Pneumonia      1676
Indeterminate_Appearance    1049
Atypical_Appearance          474
Name: target, dtype: int64

In [26]:
label_data['target'] = label_data['target'].apply(lambda x: re.sub(' ','_',x))

In [27]:
label_data[['path','target']].to_csv('./kaggle_data/image_classification.csv',index=False,header=False)

#### transferring data to storage

In [30]:
!gsutil -m cp -r /home/jupyter/kaggle_data/image_classification.csv gs://qwiklabs-gcp-03-365bf9c0599c-kaggle/

Copying file:///home/jupyter/kaggle_data/image_classification.csv [Content-Type=text/csv]...
/ [1/1 files][503.9 KiB/503.9 KiB] 100% Done                                    
Operation completed over 1 objects/503.9 KiB.                                    
